In [2]:
import numpy as np
from PIL import Image
import cv2 #opencv
import io
import time
import pandas as pd
import numpy as np
from IPython.display import clear_output
from random import randint
import os

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

#keras imports
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
from keras.callbacks import TensorBoard
from collections import deque
import random
import pickle
from io import BytesIO
import base64
import json

Using TensorFlow backend.


In [3]:
#path variables
game_url = "chrome://dino"
chrome_driver_path = "./chromedriver"
loss_file_path = "./objects/loss_df.csv"
actions_file_path = "./objects/actions_df.csv"
q_value_file_path = "./objects/q_values.csv"
scores_file_path = "./objects/scores_df.csv"

#scripts
#create id for canvas for faster selection from DOM
init_script = "document.getElementsByClassName('runner-canvas')[0].id = 'runner-canvas'"

#get image from canvas
getbase64Script = "canvasRunner = document.getElementById('runner-canvas'); \
return canvasRunner.toDataURL().substring(22)"


setting path

In [4]:
'''
* Game class: Selenium interfacing between the python and browser
* __init__():  Launch the broswer window using the attributes in chrome_options
* get_crashed() : return true if the agent as crashed on an obstacles. Gets javascript variable from game decribing the state
* get_playing(): true if game in progress, false is crashed or paused
* restart() : sends a signal to browser-javascript to restart the game
* press_up(): sends a single to press up get to the browser
* get_score(): gets current game score from javascript variables.
* pause(): pause the game
* resume(): resume a paused game if not crashed
* end(): close the browser and end the game
'''
class Game:
    def __init__(self,custom_config=True):
        chrome_options = Options()
        chrome_options.add_argument("disable-infobars")
        chrome_options.add_argument("--mute-audio")
        self._driver = webdriver.Chrome(executable_path = chrome_driver_path,chrome_options=chrome_options)
        self._driver.set_window_position(x=-10,y=0)
        self._driver.get('chrome://dino')
        self._driver.execute_script("Runner.config.ACCELERATION=0")
        self._driver.execute_script(init_script)
    def get_crashed(self):
        return self._driver.execute_script("return Runner.instance_.crashed")
    def get_playing(self):
        return self._driver.execute_script("return Runner.instance_.playing")
    def restart(self):
        self._driver.execute_script("Runner.instance_.restart()")
    def press_up(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_UP)
    def get_score(self):
        score_array = self._driver.execute_script("return Runner.instance_.distanceMeter.digits")
        score = ''.join(score_array) # the javascript object is of type array with score in the formate[1,0,0] which is 100.
        return int(score)
    def pause(self):
        return self._driver.execute_script("return Runner.instance_.stop()")
    def resume(self):
        return self._driver.execute_script("return Runner.instance_.play()")
    def end(self):
        self._driver.close()

銜接browser-javascript與python的模組，運行chrome並且連接至小恐龍遊戲的環境。
這裡可以得到狀態:撞毀、遊戲中、分數。指令:重啟遊戲、跳躍、暫停、恢復遊戲、結束。

In [5]:
class DinoAgent:
    def __init__(self,game): #takes game as input for taking actions
        self._game = game; 
        self.jump(); #to start the game, we need to jump once
    def is_running(self):
        return self._game.get_playing()
    def is_crashed(self):
        return self._game.get_crashed()
    def jump(self):
        self._game.press_up()
    def duck(self):
        self._game.press_down()

這裡為DinoAgent的模組，可知Agent是否撞毀，並有跳躍與蹲下兩種行為。我們在遊戲中只考慮跳躍與不跳躍兩種。

In [6]:
class Game_sate:
    def __init__(self,agent,game):
        self._agent = agent
        self._game = game
        self._display = show_img() #display the processed image on screen using openCV, implemented using python coroutine 
        self._display.__next__() # initiliaze the display coroutine 
    def get_state(self,actions):
        actions_df.loc[len(actions_df)] = actions[1] # storing actions in a dataframe
        score = self._game.get_score() 
        reward = 0.1
        is_over = False #game over
        if actions[1] == 1:
            self._agent.jump()
        image = grab_screen(self._game._driver) 
        self._display.send(image) #display the image on screen
        if self._agent.is_crashed():
            scores_df.loc[len(loss_df)] = score # log the score when game is over
            self._game.restart()
            reward = -1
            is_over = True
        return image, reward, is_over #return the Experience tuple

為了確認Game的state，這裡使用OpenCV的截圖來獲取環境的狀態(OpenCV中影像處理的速度有經過特別優化)，這裡的get_state方法為輸入一個action，然後回傳下一狀態的影像、reward(如果撞毀=-1，沒撞毀=0.1)、is_over(是否撞毀)。OpenCV平均可以達到5FPS的擷取速度，這對於每個跳Action後的State轉換已經足夠表示。

In [7]:
def save_obj(obj, name ):
    with open('objects/'+ name + '.pkl', 'wb') as f: #dump files into objects folder
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(name ):
    with open('objects/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def grab_screen(_driver):
    image_b64 = _driver.execute_script(getbase64Script)
    screen = np.array(Image.open(BytesIO(base64.b64decode(image_b64))))
    image = process_img(screen)#processing image as required
    return image

def process_img(image):
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #RGB to Grey Scale
    image = image[:300, :500] #Crop Region of Interest(ROI)
    image = cv2.resize(image, (80,80))
    return  image

def show_img(graphs = False):
    """
    Show images in new window
    """
    while True:
        screen = (yield)
        window_title = "logs" if graphs else "game_play"
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)        
        imS = cv2.resize(screen, (800, 400)) 
        cv2.imshow(window_title, screen)
        if (cv2.waitKey(1) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break

儲存object方法，後面會儲存的資料有:行為，格式為(S,a,r,St+1,是否結束)、時間(為了辨認FPS的更新狀態)、epsilon(隨機行為的機率隨時間遞減的參數)，螢幕截圖(單純擷取chrome中的遊戲影像)，影像處理(只保留有興趣的影像區域，在小恐龍的遊戲中其實只在乎仙人掌與螢幕左邊界的距離。size為80*80)

In [8]:
#Intialize log structures from file if exists else create new
loss_df = pd.read_csv(loss_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns =['loss'])
scores_df = pd.read_csv(scores_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns = ['scores'])
actions_df = pd.read_csv(actions_file_path) if os.path.isfile(actions_file_path) else pd.DataFrame(columns = ['actions'])
q_values_df =pd.read_csv(actions_file_path) if os.path.isfile(q_value_file_path) else pd.DataFrame(columns = ['qvalues'])

使用DateFrame儲存loss,score,action,q_values。

In [9]:
#game parameters
ACTIONS = 2 # possible actions: jump, do nothing
GAMMA = 0.99 # decay rate of past observations original 0.99
OBSERVATION = 100. # timesteps to observe before training
EXPLORE = 100000  # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 16 # size of minibatch
FRAME_PER_ACTION = 1
LEARNING_RATE = 1e-4
img_rows , img_cols = 80,80
img_channels = 4 #We stack 4 frames

In [12]:
# training variables saved as checkpoints to filesystem to resume training from the same step
def init_cache():
    """initial variable caching, done only once"""
    save_obj(INITIAL_EPSILON,"epsilon")
    t = 0
    save_obj(t,"time")
    D = deque()
    save_obj(D,"D")

In [13]:
'''Call only once to init file structure
'''
init_cache()

如果想要執行訓練，第一次需要執行init_cache()，他會創建所需的檔案。在這之前須建立object空白資料夾以存放。

In [14]:
def buildmodel():
    print("Now we build the model")
    model = Sequential()
    model.add(Conv2D(32, (8, 8), padding='same',strides=(4, 4),input_shape=(img_cols,img_rows,img_channels)))  #80*80*4
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4),strides=(2, 2),  padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3),strides=(1, 1),  padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    
    #create model file if not present
    if not os.path.isfile(loss_file_path):
        model.save_weights('model.h5')
    print("We finish building the model")
    return model

# 建立模型
模型輸入為80x80x4，即每四張影像為一組State，再經過幾次捲積與池化，再展平成512的特徵向量，最後輸出(0,1)代表不跳與跳，這裡的類神經網路用來替代傳統的Q-table，由於影像為格式的state很難窮舉到Q-table中，因此使用類神經網路，好處是可以舉一反三，State的pixel值即使不盡相同，但是仍可以訓練出其特性。這個方法即為DQN，為一種off-policy learning。

In [15]:
''' 
main training module
Parameters:
* model => Keras Model to be trained
* game_state => Game State module with access to game environment and dino
* observe => flag to indicate wherther the model is to be trained(weight updates), else just play
'''
def trainNetwork(model,game_state,observe=False):
    last_time = time.time()
    # store the previous observations in replay memory
    D = load_obj("D") #load from file system
    # get the first state by doing nothing
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] =1 #0 => do nothing,
                     #1=> jump
    
    x_t, r_0, terminal = game_state.get_state(do_nothing) # get next step after performing the action
    

    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2) # stack 4 images to create placeholder input
    

    
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*20*40*4
    
    initial_state = s_t 

    if observe :
        OBSERVE = 999999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        #epsilon = load_obj("epsilon") 
        epsilon = 0
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)

    t = load_obj("time") # resume from the previous time step stored in file system
    while (True): #endless running
        
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0 #reward at 4
        a_t = np.zeros([ACTIONS]) # action at t
        
        #choose an action epsilon greedy
        if t % FRAME_PER_ACTION == 0: #parameter to skip frames for actions
            if  random.random() <= epsilon: #randomly explore an action
                print("----------Random Action----------")
                action_index = random.randrange(ACTIONS)
                a_t[0] = 1
            else: # predict the output
                q = model.predict(s_t)       #input a stack of 4 images, get the prediction
                max_Q = np.argmax(q)         # chosing index with maximum q value
                action_index = max_Q 
                a_t[action_index] = 1        # o=> do nothing, 1=> jump
                
        #We reduced the epsilon (exploration parameter) gradually
        #if epsilon >= FINAL_EPSILON and t > OBSERVE:
            #epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE 
            #epsilon=0
        #run the selected action and observed next state and reward
        x_t1, r_t, terminal = game_state.get_state(a_t)
        print('fps: {0}'.format(1 / (time.time()-last_time))) # helpful for measuring frame rate
        last_time = time.time()
        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x20x40x1
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3) # append the new image to input stack and remove the first one
        
        
        # store the transition in D
        D.append((s_t, action_index, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #only train if done observing
        if t > OBSERVE: 
            
            #sample a minibatch to train on
            minibatch = random.sample(D, BATCH)
            inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))   #32, 20, 40, 4
            targets = np.zeros((inputs.shape[0], ACTIONS))                         #32, 2

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                state_t = minibatch[i][0]    # 4D stack of images
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]   #reward at state_t due to action_t
                state_t1 = minibatch[i][3]   #next state
                terminal = minibatch[i][4]   #wheather the agent died or survided due the action
                

                inputs[i:i + 1] = state_t    

                targets[i] = model.predict(state_t)  # predicted q values
                Q_sa = model.predict(state_t1)      #predict q values for next step
                
                if terminal:
                    targets[i, action_t] = reward_t # if terminated, only equals reward
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            loss += model.train_on_batch(inputs, targets)
            loss_df.loc[len(loss_df)] = loss
            q_values_df.loc[len(q_values_df)] = np.max(Q_sa)
        s_t = initial_state if terminal else s_t1 #reset game to initial frame if terminate
        t = t + 1
        
        # save progress every 1000 iterations
        if t % 20000 == 0:
            print("Now we save model")
            game_state._game.pause() #pause game while saving to filesystem
            model.save_weights("model.h5", overwrite=True)
            save_obj(D,"D") #saving episodes
            save_obj(t,"time") #caching time steps
            save_obj(epsilon,"epsilon") #cache epsilon to avoid repeated randomness in actions
            loss_df.to_csv("./objects/loss_df.csv",index=False)
            scores_df.to_csv("./objects/scores_df.csv",index=False)
            actions_df.to_csv("./objects/actions_df.csv",index=False)
            q_values_df.to_csv(q_value_file_path,index=False)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)
            clear_output()
            game_state._game.resume()
        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state,             "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t,             "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

    print("Episode finished!")
    print("************************")


# Training
這裡寫下訓練的過程，我們先觀察100次，然後到累積10萬筆之前都是EXPLORE狀態，但仍會訓練神經網路，這是因為epsilon(隨機動作的機率)在10萬次以後才會收斂到最低值，後面的狀態我們稱為訓練狀態。
訓練方式是這樣的:使用當前state預測下一步的action並且概率地(取決於epsilon)執行，這時候觀察St+1與reward並記錄下來，然後從replay buffer中隨機取出32筆資料出來訓練，得到32組(S,A,r,St+1,terminal)，如果恐龍在此State,S採取Action,A，陣亡了，那麼reward就不更新，但如果沒有陣亡，則reward=reward+GAMMA*max(Q_sa)，意思是此State的Action會讓你的恐龍活著，所以是好棒棒的，所以多給一點reward，然後在拿其S,A,r進行supervised learning，Ptrue為更新過的reward，在進行反向傳播法更新參數，如此一來網路就會更像理想的Q_table，

In [16]:
#main function
def playGame(observe=False):
    game = Game()
    dino = DinoAgent(game)
    game_state = Game_sate(dino,game)    
    model = buildmodel()
    try:
        trainNetwork(model,game_state,observe=observe)
    except StopIteration:
        game.end()

In [ ]:
playGame(observe=False);

Now we build the model
We finish building the model
fps: 3.941933531449538
TIMESTEP 1 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 21.724715900262087
TIMESTEP 2 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 21.72381573904306
TIMESTEP 3 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 21.03789455732837
TIMESTEP 4 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 21.722915652417107
TIMESTEP 5 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 20.604451693087643
TIMESTEP 6 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 21.72381573904306
TIMESTEP 7 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 10.74498270782631
TIMESTEP 8 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 5.693970584641784
TIMESTEP 9 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q

TIMESTEP 78 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 19.985724156005052
TIMESTEP 79 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 23.239201258837348
TIMESTEP 80 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 21.037683514653587
TIMESTEP 81 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 21.037683514653587
TIMESTEP 82 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 21.261635009707458
TIMESTEP 83 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 21.037366958580357
TIMESTEP 84 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 22.206301388719762
TIMESTEP 85 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 21.261635009707458
TIMESTEP 86 / STATE observe / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 21.490185630184502
TIMESTEP 87 / STATE

TIMESTEP 144 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  4.9616914 / Loss  0.6733856201171875
fps: 12.97786743979529
TIMESTEP 145 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  3.2446015 / Loss  0.8149788975715637
fps: 10.68756800281312
TIMESTEP 146 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  2.9308681 / Loss  0.9515091180801392
fps: 10.518788997451999
TIMESTEP 147 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  7.0303535 / Loss  0.9452688097953796
fps: 10.981319495534745
TIMESTEP 148 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  3.9925394 / Loss  0.29657119512557983
fps: 10.574399907222292
TIMESTEP 149 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  4.7465076 / Loss  0.8415642976760864
fps: 12.491226398236941
TIMESTEP 150 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  3.5524912 / Loss  0.5073165893554688
fps: 13.062623796295142
TIMESTEP 151 / STATE explore / EPSILON 0 / ACTION 1 / RE

TIMESTEP 205 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  8.145391 / Loss  0.4965343475341797
fps: 10.745092815092251
TIMESTEP 206 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.079951 / Loss  0.5956228971481323
fps: 16.248678970449227
TIMESTEP 207 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  8.908698 / Loss  0.22969657182693481
fps: 15.613916746703595
TIMESTEP 208 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  7.3131976 / Loss  0.2867915630340576
fps: 12.97778712897327
TIMESTEP 209 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  7.7353578 / Loss  0.9491228461265564
fps: 15.9886555102352
TIMESTEP 210 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  7.129387 / Loss  0.4151580035686493
fps: 15.736882192048085
TIMESTEP 211 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  6.7158484 / Loss  0.6363164782524109
fps: 15.86161985546324
TIMESTEP 212 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0

TIMESTEP 266 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  8.945001 / Loss  0.1819562017917633
fps: 10.745120342261332
TIMESTEP 267 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  10.0749445 / Loss  0.3308781683444977
fps: 10.68754076973255
TIMESTEP 268 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  8.008649 / Loss  0.1795467883348465
fps: 10.803156745173188
TIMESTEP 269 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  6.909386 / Loss  0.6657708883285522
fps: 10.355283428797156
TIMESTEP 270 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.537019 / Loss  0.2004561573266983
fps: 10.861935998466905
TIMESTEP 271 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  6.672122 / Loss  0.11134923994541168
fps: 10.409251997816051
TIMESTEP 272 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  7.287189 / Loss  0.6676126718521118
fps: 11.041884513523616
TIMESTEP 273 / STATE explore / EPSILON 0 / ACTION 1 / REWARD

...

TIMESTEP 10464 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.670721 / Loss  0.3227836489677429
fps: 9.043385266861867
TIMESTEP 10465 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.391171 / Loss  0.29811978340148926
fps: 9.2527807069018
TIMESTEP 10466 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.051388 / Loss  0.22259989380836487
fps: 9.125948102923834
TIMESTEP 10467 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  11.842747 / Loss  0.2211419939994812
fps: 10.630736671355612
TIMESTEP 10468 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  5.57824 / Loss  0.2670024037361145
fps: 10.518815377323238
TIMESTEP 10469 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  10.368662 / Loss  0.30034756660461426
fps: 9.33914631159404
TIMESTEP 10470 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  6.024249 / Loss  0.4273843467235565
fps: 10.46198513877214
TIMESTEP 10471 / STATE explore / EPSILON 0 / ACTION 

TIMESTEP 10526 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  7.747508 / Loss  0.6898525953292847
fps: 9.21003236672332
TIMESTEP 10527 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  11.657573 / Loss  0.3277250826358795
fps: 9.002564922590851
TIMESTEP 10528 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  5.364592 / Loss  0.39099442958831787
fps: 10.687649702887546
TIMESTEP 10529 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  5.340062 / Loss  0.6667595505714417
fps: 9.12596795916458
TIMESTEP 10530 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  9.702788 / Loss  0.6649786233901978
fps: 9.167797440022383
TIMESTEP 10531 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  11.528002 / Loss  0.396634578704834
fps: 9.12590839070156
TIMESTEP 10532 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  6.236967 / Loss  0.718703031539917
fps: 9.252699059796253
TIMESTEP 10533 / STATE explore / EPSILON 0 / ACTION 1 / 

fps: 10.80468735912291
TIMESTEP 10587 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  10.307363 / Loss  0.6656014919281006
fps: 9.2527807069018
TIMESTEP 10588 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  4.4964156 / Loss  0.3455080986022949
fps: 12.811494688227352
TIMESTEP 10589 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  6.69516 / Loss  0.2993153929710388
fps: 10.630736671355612
TIMESTEP 10590 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  2.3366704 / Loss  0.269317626953125
fps: 10.35530899493629
TIMESTEP 10591 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  0.7340808 / Loss  0.29425105452537537
fps: 10.921187755824732
TIMESTEP 10592 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  7.917675 / Loss  0.2485649287700653
fps: 9.16775736274497
TIMESTEP 10593 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  0.8074124 / Loss  0.4486296474933624
fps: 10.745092815092251
TIMESTEP 10594 / STATE expl

fps: 9.471889001300767
TIMESTEP 10648 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  6.265021 / Loss  0.3508158028125763
fps: 10.74501023443124
TIMESTEP 10649 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  9.259972 / Loss  0.6110091209411621
fps: 9.210072814468033
TIMESTEP 10650 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  4.92197 / Loss  0.1327207088470459
fps: 8.9622454844796
TIMESTEP 10651 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  8.507721 / Loss  0.17342060804367065
fps: 9.25265823678382
TIMESTEP 10652 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  9.684368 / Loss  0.33785584568977356
fps: 10.981233243967829
TIMESTEP 10653 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  5.121502 / Loss  0.19950860738754272
fps: 9.043385266861867
TIMESTEP 10654 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  5.715042 / Loss  0.17962898313999176
fps: 9.25265823678382
TIMESTEP 10655 / STATE explore /

TIMESTEP 10710 / STATE explore / EPSILON 0 / ACTION 0 / REWARD -1 / Q_MAX  6.650728 / Loss  0.22147053480148315
fps: 8.360916644074825
TIMESTEP 10711 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  4.222417 / Loss  0.33078768849372864
fps: 9.3401029697326
TIMESTEP 10712 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  10.240317 / Loss  0.30792051553726196
fps: 10.74506528806421
TIMESTEP 10713 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  11.073266 / Loss  0.2758011817932129
fps: 10.574506546188891
TIMESTEP 10714 / STATE explore / EPSILON 0 / ACTION 0 / REWARD 0.1 / Q_MAX  5.3996887 / Loss  0.18151968717575073
fps: 9.001850031549262
TIMESTEP 10715 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  4.7578344 / Loss  0.4288857579231262
fps: 6.489305181011262
TIMESTEP 10716 / STATE explore / EPSILON 0 / ACTION 1 / REWARD 0.1 / Q_MAX  8.4930935 / Loss  0.16255277395248413
fps: 10.68756800281312
TIMESTEP 10717 / STATE explore / EPSILON 0 / AC